# Overview

I exeprimented with context free feature grammar in order to parse a sentence into an xXpath query. The elements of 
the Xpath query are set by the terminal nodes/words as well as by non-terminal nodes/phrases. The combination of the elements also happens at higher nodes. 

# Simple Grammar

I defined a simple grammar and draw the parse trees.

In [2]:
from nltk import grammar, parse
import nltk,urllib.request, gzip, io
from nltk import grammar, parse
from nltk.tokenize import word_tokenize
import re
from lxml import etree

In [3]:
import re
g = """
% start S
S[SEM=(?np + '[' + ?vp+']')] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?p)] -> TV[SEM=?v] NP[SEM=?p] | CMPLX-V[SEM=?v] PP[SEM=?p] | TV[SEM=?v] CONJP[SEM=?p]
CONJP[SEM=(?spec + ?compl)] -> NP[SEM=?spec] CONJP[SEM=?compl] | CONJ[SEM=?spec] CONJP[SEM=?compl] | CONJ[SEM=?spec] NP[SEM=?compl] | VP[SEM=?spec] CONJP[SEM=?compl] | CONJ[SEM=?spec] VP[SEM=?compl]
NP[SEM=(?det + ?n)] -> Art[SEM=?det] NBAR[SEM=?n] | Int[SEM=?det] N[SEM=?n]
NBAR[SEM=(?np + ?num)] -> N[SEM=?np] PP[SEM=?num] | N[SEM=?np] AP[SEM=?num]
PP[SEM=(?p + ?num)] -> P[SEM=?p] NUM[SEM=?num] | P[SEM=?p] ADVP[SEM=?num]
ADVP[SEM=(?adv + ?num)] -> CMPLX-ADV[SEM=?adv] NUM[SEM=?num] 
AP[SEM=(?a + ?num)] -> CMPLX-A[SEM=?a] NUM[SEM=?num]
CMPLX-A[SEM=?a] -> A[SEM=?a] CONJ
CMPLX-ADV[SEM=?adv] -> ADV[SEM=?adv] CONJ | P ADV[SEM=?adv]
CMPLX-V[SEM=?part] -> AUX PART[SEM=?part]

PART[SEM='discoveryyear'] -> 'discovered'
Int[SEM='.//'] -> 'which' | 'what'
AUX -> 'were'
TV[SEM=''] -> 'have' | 'possess'
Art[SEM=''] -> 'a'
N[SEM='mass'] -> 'mass'
N[SEM='radius'] -> 'radius'
N[SEM='planet'] -> 'planets'
P[SEM=''] -> 'of' |'at'
P[SEM='<'] -> 'before'
P[SEM='>'] -> 'after'
CONJ[SEM=''] -> 'than'
CONJ[SEM=' and '] -> 'and'
A[SEM='>'] -> 'bigger' | 'larger' | 'greater' 
A[SEM='<'] -> 'smaller'
ADV[SEM='<'] -> 'less' | 'most'
ADV[SEM='>'] -> 'more' | 'least'
NUM[SEM="=#NUM#"] -> '#NUM#'
"""
sents = ['what planets have a mass of 19.4','which planets have a mass of 19.4',
'what planets have a radius of 0.188','which planets have a mass of at least 19.4',
'which planets have a mass of at most 0.001','which planets have a mass smaller than 0.001',
'which planets have a mass greater than 19.4','what planets were discovered before 2001',
'what planets were discovered after 2021','which planets have a mass smaller than 0.001',
'which planets have a mass smaller than 0.001 and a radius of less than 0.188',
'what planets have a mass larger than 0.001 and a radius of at least 0.188 and a mass of at least 0.188',
'what planets have a mass larger than 0.001 and were discovered before 2020']
sents = [re.sub(r'\d+\.*\d*','#NUM#',s).split() for s in sents]

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=0)

trees = [list(parser.parse(s))[0] for _,s in enumerate(sents)]

from nltk.draw.tree import draw_trees
draw_trees(trees[-1])